In [1]:
import pandas as pd
pd.options.display.max_rows = 999

import numpy as np
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, make_scorer

import warnings
warnings.simplefilter('ignore')

In [11]:
#удаляем служебные колонки, признак pdb_chain - строка, его хэшируем
def preprocess_data(df):
    for column in ['DSSR', 'Id', 'index']:
        if column in df.columns:
            df.drop(column, axis=1, inplace=True)
    df.dropna(inplace=True)
    
    df['pdb_chain'] = df['pdb_chain'].apply(hash)
    
    return df

In [36]:
df_train = pd.read_table('./train.csv')

In [37]:
preprocess_data(df_train)
X_train = df_train.loc[:, df_train.columns != 'mg']
Y_train = df_train['mg']

In [43]:
#снижаем размерность признаков, на самом деле можно было и побольше удалить

from sklearn.feature_selection import SelectPercentile
from sklearn.feature_selection import f_classif

selector_f = SelectPercentile(f_classif, percentile=70)
X_train = selector_f.fit_transform(X_train, Y_train)

In [47]:
#фиксируем learning rate и подбираем под него число деревьев

#scale_pos_weight = 6, т.к. учебная выборка была несбалансированна 
#(кол-во случаев, когда Mg не присоединился) / (кол-во случаев когда присоединился) ~ 6

#для остальных параметров выбраны более менее стандартные значения

clf = XGBClassifier(learning_rate=0.2, n_estimators=4000, max_depth=6, min_child_weight=1, 
                    subsample=0.8, colsample_bytree=0.7, scale_pos_weight=6, objective= 'binary:logistic',
                    nthread=8)

cvresult = xgb.cv(clf.get_xgb_params(), xgb.DMatrix(X_train, label=Y_train), 
                  num_boost_round=clf.get_params()['n_estimators'], nfold=5, early_stopping_rounds=50)

#в cv_result получили n_estimator ~ 3000

In [51]:
clf = XGBClassifier(learning_rate=0.2, n_estimators=3000, max_depth=6, min_child_weight=1, 
                    subsample=0.8, colsample_bytree=0.7, scale_pos_weight=6, objective= 'binary:logistic',
                    nthread=8)
clf.fit(X_train, Y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.7, gamma=0, learning_rate=0.2, max_delta_step=0,
       max_depth=6, min_child_weight=1, missing=None, n_estimators=3000,
       n_jobs=1, nthread=8, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=6, seed=None,
       silent=True, subsample=0.8)

In [52]:
Y_pred_train = clf.predict(X_train)

In [53]:
print(f1_score(Y_train, Y_pred_train))

0.9751276277566736


In [54]:
#зафиксировав learning_rate и подходящее число деревьев ищем глубину

param_test1 = {'max_depth': [4, 5, 6, 7, 8]}

gsearch1 = GridSearchCV(estimator = XGBClassifier(learning_rate=0.2, n_estimators=3000, max_depth=6, min_child_weight=1, 
                                                    subsample=0.8, colsample_bytree=0.7, scale_pos_weight=6, gamma=0, 
                                                      objective= 'binary:logistic', nthread=8),
                        param_grid = param_test1, scoring=make_scorer(f1_score), iid=False, cv=5, n_jobs=8)
gsearch1.fit(X_train, Y_train)

#совсем не угадал с range и все считал зря

GridSearchCV(cv=5, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.7, gamma=0, learning_rate=0.2, max_delta_step=0,
       max_depth=6, min_child_weight=1, missing=None, n_estimators=3000,
       n_jobs=1, nthread=8, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=6, seed=None,
       silent=True, subsample=0.8),
       fit_params=None, iid=False, n_jobs=8,
       param_grid={'max_depth': [4, 5, 6, 7, 8]}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score='warn',
       scoring=make_scorer(f1_score), verbose=0)

In [56]:
print(gsearch1.grid_scores_)
print(gsearch1.best_params_)
print(gsearch1.best_score_)

[mean: 0.20397, std: 0.03689, params: {'max_depth': 4}, mean: 0.16909, std: 0.04657, params: {'max_depth': 5}, mean: 0.14382, std: 0.05216, params: {'max_depth': 6}, mean: 0.13017, std: 0.06159, params: {'max_depth': 7}, mean: 0.12099, std: 0.06833, params: {'max_depth': 8}]
{'max_depth': 4}
0.20396999262039933


In [58]:
param_test2 = {'max_depth': [3]}

gsearch2 = GridSearchCV(estimator = XGBClassifier(learning_rate=0.2, n_estimators=3000, max_depth=6, min_child_weight=1, 
                                                    subsample=0.8, colsample_bytree=0.7, scale_pos_weight=6, gamma=0, 
                                                      objective= 'binary:logistic', nthread=8),
                        param_grid = param_test2, scoring=make_scorer(f1_score), iid=False, cv=5, n_jobs=8)
gsearch2.fit(X_train, Y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.7, gamma=0, learning_rate=0.2, max_delta_step=0,
       max_depth=6, min_child_weight=1, missing=None, n_estimators=3000,
       n_jobs=1, nthread=8, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=6, seed=None,
       silent=True, subsample=0.8),
       fit_params=None, iid=False, n_jobs=8, param_grid={'max_depth': [3]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=make_scorer(f1_score), verbose=0)

In [60]:
print(gsearch2.grid_scores_)
print(gsearch2.best_params_)
print(gsearch2.best_score_)

[mean: 0.23463, std: 0.03032, params: {'max_depth': 3}]
{'max_depth': 3}
0.23463357039832702


In [61]:
param_test3 = {'max_depth': [2]}

gsearch3 = GridSearchCV(estimator = XGBClassifier(learning_rate=0.2, n_estimators=3000, max_depth=6, min_child_weight=1, 
                                                    subsample=0.8, colsample_bytree=0.7, scale_pos_weight=6, gamma=0, 
                                                      objective= 'binary:logistic', nthread=8),
                        param_grid = param_test2, scoring=make_scorer(f1_score), iid=False, cv=5, n_jobs=8)
gsearch3.fit(X_train, Y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.7, gamma=0, learning_rate=0.2, max_delta_step=0,
       max_depth=6, min_child_weight=1, missing=None, n_estimators=3000,
       n_jobs=1, nthread=8, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=6, seed=None,
       silent=True, subsample=0.8),
       fit_params=None, iid=False, n_jobs=8, param_grid={'max_depth': [3]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=make_scorer(f1_score), verbose=0)

In [62]:
print(gsearch3.grid_scores_)
print(gsearch3.best_params_)
print(gsearch3.best_score_)

[mean: 0.23463, std: 0.03032, params: {'max_depth': 3}]
{'max_depth': 3}
0.23463357039832702


In [ ]:
# Последние два результата странные, была выбрана глубина 3, т.к. 2 как-то слишком мало

In [75]:
clf1 = XGBClassifier(learning_rate=0.2, n_estimators=3000, max_depth=3, min_child_weight=1, 
                    subsample=0.8, colsample_bytree=0.7, scale_pos_weight=6, objective= 'binary:logistic',
                    nthread=8, gamma=1)
clf1.fit(X_train, Y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.7, gamma=1, learning_rate=0.2, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=3000,
       n_jobs=1, nthread=8, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=6, seed=None,
       silent=True, subsample=0.8)

In [76]:
df_test = pd.read_csv('./test.csv')
df_test = preprocess_data(df_test)
df_test = selector_f.transform(df_test)

In [77]:
Y_pred = clf1.predict(df_test)

In [78]:
df_out = pd.DataFrame({"Id" : range(len(Y_pred)), "mg" : Y_pred})
df_out.to_csv("sample_submission.csv", index=False)

Запуск с вышеприведнными параметрами получил результат $0.37891$ на PublicLeaderboard и шаги по дальнейшему улучшению не предпринимались.

Что предполаголось делать: таким же образом потюнить min_child_weight, gammа (была посылка с gamma=1, но она ухудшила score),  subsample and colsample_bytree. Уменьшить learning_rate и подобрать к нему еще раз кол-во деревьев. Не выкидывать все строки с Nan, а выкинуть ненужные столбцы, в остальных заменить средним / ... .

Почему этого не было: все достаточно долго считается, даже на google.cloud. Признаков так много, что не очень понятно, что с ними делать: ручками их как-то долго перебирать и графики смотреть. Возможно данных слишком много для первого учебного kaggle?